# Homework: Implementing and Evaluating a 1D Diffusion Model

## Preliminaries

We encourage using LLMs and coding agents for this homework to grasp concepts and implement code. Otherwise, it may be too challenging. **Please review the ISIS page on recommended AI tool usage.**

For this homework, we recommend setting up a repository (e.g., using GitHub or GitLab).
Initialize the package with a `pyproject.toml` and manage dependencies via the [Astral UV](https://docs.astral.sh/uv/) package manager (which has numerous advantages to using `anaconda`).

We encourage to try out different concepts taught in the course and think about tradeoffs of different solution approaches.
Your code should be performant, readable, maintainable, and extendable (e.g., toward more model configurations and more complex data).
If you decide **not** to use some patterns from the course (e.g., because they would make the code too complex), think about trade-offs (e.g., in what scenarios could such a pattern become useful?).

You should be able to complete this homework on an average laptop without GPUs or high-performance CPUs.
However, you may optionally use free compute offerings (for GPUs, but high-perf CPUs may also be helpful) that are available online, e.g., via Google Colab.

## Background: Diffusion Models

This assignment uses the [`tinydiffusion` repository](https://github.com/tanelp/tiny-diffusion), which implements a Denoising Diffusion Probabilistic Model (DDPM). You may read [this blog](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/) for background on diffusion models (The blog post may look at bit scary at first. However, note that you don't need to understand all math derivations.).

### Forward Process: Adding Noise

The model first learns a data distribution by gradually corrupting data with Gaussian noise over $T$ timesteps. The process is defined by a *noise schedule* $\beta_t$ for $t=1, ..., T$. From this, we define:

- $\alpha_t = 1 - \beta_t$
- $\bar{\alpha}_t = \prod_{i=1}^t \alpha_i$ (the cumulative product)

The noisy sample $x_t$ at timestep $t$ is sampled from the conditional distribution $q(x_t | x_0) = \mathcal{N}(x_t; \sqrt{\bar{\alpha}_t} x_0, (1 - \bar{\alpha}_t)I)$.

**Intuition:** This equation constructs $x_t$ by mixing the original data $x_0$ and pure Gaussian noise $\epsilon \sim \mathcal{N}(0, I)$. The term $\sqrt{\bar{\alpha}_t}$ scales down the data, while $(1 - \bar{\alpha}_t)$ scales the noise. As $t$ increases towards $T$, $\bar{\alpha}_t$ approaches 0, so the signal from $x_0$ vanishes and $x_T$ becomes nearly indistinguishable from pure noise.

### Reverse Process: Denoising

The model learns to reverse this process. A neural network, $\epsilon_\theta(x_t, t)$ ("$\theta$" are parameters, i.e., neural net weights), is trained to predict the noise $\epsilon$ (no "$\theta$" here, i.e., this is the true noise) that was added to an original sample $x_0$ to produce the noisy version $x_t$. The training objective is the mean squared error (MSE) between the true and predicted noise:

$L(\theta) = \mathbb{E}_{t, x_0, \epsilon} [\|\epsilon - \epsilon_\theta(\sqrt{\bar{\alpha}_t}x_0 + \sqrt{1-\bar{\alpha}_t}\epsilon, t)\|^2]$

This loss function directly trains the network to estimate the noise component from a given noisy sample $x_t$ and timestep $t$. Timesteps $t$ are (somewhat) randomly and independently sampled during training. For each sampled timestep, we corrupt the ground-truth data using using the forward process and pass this corrupted sample to our neural net $\epsilon_\theta$. The neural net learns to estimate the noise that was added via the forward process (intuition: if you can identify the noise from a noisy "image", you can de-noise the "image"), i.e., we compare the neural net output to the true noise and minimize their distance. This is done in parallel for many sampled timesteps.

**Note (optional):** This MSE objective is a practical simplification (validated experimentally) of the full Evidence Lower Bound (ELBO) objective used in the original DDPM paper.
The ELBO is a theoretically derived loss function, which is often found in the [VAE literature](https://arxiv.org/abs/1906.02691) (note: diffusion models are a special kind of hierarchical VAE).

### Generating New Samples (inference/test time)

Generating samples reverses the diffusion process. We start with pure noise $x_T \sim \mathcal{N}(0, I)$ and iteratively denoise it by looping backwards from $t=T$ down to $1$ ($x_{t-1}$ for $t=1$ is $x_0$).

In each step, our network $\epsilon_\theta(x_t, t)$ predicts the noise in the current sample $x_t$. This is used to compute the mean of the next, cleaner sample $x_{t-1}$:

$\mu_\theta(x_t, t) = \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{\beta_t}{\sqrt{1-\bar{\alpha}_t}} \epsilon_\theta(x_t, t) \right)$

We then obtain $x_{t-1}$ by sampling from $\mathcal{N}(x_{t-1}; \mu_\theta(x_t, t), \sigma_t^2 I)$. After all steps, the result $\hat{x}_0$ is our generated sample.

A key distinction to remember:
During **training**, we sample timesteps *randomly* and independently.
For **generation**, the process must be *sequential*. We start from pure noise at the final timestep $T$ and work backwards one step at a time to $t=1$.

Your focus for this homework is on correctly implementing this process; a deep understanding of the underlying mathematical derivations is not required.

---

## Your Tasks

### Part 1: Data Generation

1. **Implement a Data Source**:  Create a "complicated" 1D data distribution that has a tractable [probability density function (PDF)](https://en.wikipedia.org/wiki/Probability_density_function). For example, you can define the data as a mixture of Gaussian, where some components may be differently weighted and/or overlapping.
    - Intuition: A PDF is a smooth curve that acts like a continuous histogram. The x-axis shows all possible values a data point can take (e.g., $-\infty$ to $\infty$ for our 1D data). The height of the curve at a certain point (the y-axis) indicates how common data is. A high peak means that values in that region are frequent; a low valley means they are rare. PDFs are normalized.
    - We typically don't know the true PDF in practice, e.g., the true distribution generating images. In this exercise, we only show samples from the PDF to the model (not the PDF itself) and will later use the PDF to evaluate model performance.
2. **Create Datasets**: Generate training, validation, and test sets from your distribution (i.e., you sample from the above distribution, split these samples into `train`, `val`, and `test`, and save the samples to disk). The best practice is to train on `train`, tune hyperparameters on `val`, and evaluate on `test` (for this HW, you may ignore `test`).
    * **Hint**: You can use standard library tools like `torch.utils.data.Dataset` and `DataLoader`, or (advanced) you can implement your own data loading abstractions using features like iterators (`__iter__`, `__next__`).

### Part 2: Model Adaptation and Training

1. **Adapt the Model**: Modify the `tinydiffusion` codebase to work with 1D data (you can remove the 2D functionality completely).
2. **Train the Model**: Train your adapted model on your 1D dataset.

### Part 3: Hyperparameter Search

1. Implement a system that allows to compute results for varying hyperparameters. Each run should create a "run directory" that collects relevant artifacts (e.g., model checkpoints). It should be possible to deploy training and eval runs independently (e.g., train a model on one day, and run evals on the trained model on another day). You may use `dataclasses` to cleanly define and manage experiment configurations.

### Part 4: Evaluation

#### 1.Target Analysis: How well can we generate the data?

* **Quantitative**: For each trained model, compute and plot its validation loss against its sample quality. Does a lower loss correlate with higher quality?
    1. Validation Loss: The final MSE loss on the validation set (averaged over samples).
    2. Log-likelihood: Generate a set of samples $\{\hat{x}_i\}_{i=1}^N$ from the model. Calculate their average log-likelihood under the true PDF, $p(x)$, of your data source: $\frac{1}{N} \sum_{i=1}^N \log p(\hat{x}_i)$.
* **Qualitative**: For your best model, plot a histogram of its generated samples overlaid with the true data PDF curve.

#### 2. Diagnostic Analysis: Forward vs. Reverse Trajectory Alignment

* **Quantitative:** At each timestep `t`, compute the 1D Wasserstein distance (mean absolute difference of sorted samples) between the samples from the forward and reverse pass. Plot this distance vs. `t` to see if the reverse process successfully tracks the forward process (i.e., the distance trends to zero).

* **Visual:** For a few key timesteps (e.g., $t=T, T/2, 1$), plot the histograms of the forward and reverse sample distributions. This provides a direct visual check of their alignment.


### Part 5: Report

Summarize your findings in a short report, focusing on:

**Results:**

1. Target: Can we generate the data truthfully? Do the validation loss, the log-likelihood, and the histogram measure agree?
2. Diagnostics: can we explain good/poor performance in (1) using our trajectory alignment diagnostics?
3. Hyperparameters: What is the impact of `num_timesteps` on sample quality (and trajectory alignment)? What is the smallest number of timesteps that still produces meaningful results?

**Coding:**

1. What concepts/patterns from PyML have you found useful?
2. Assume you'd want to significantly scale your implementation (e.g., train/eval on multiple GPUs; larger hp search; more evals; additional implementation of baselines/model variations). Would this work with your current code? What concepts/patterns from PyML could prove useful here?
3. Would there be different ways of implementing your ideas? What are trade-offs?

In [ ]:
testtest = []